In [1]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
from db_client import RDSClient

def main():
    # 1. DB 클라이언트 인스턴스 생성 (이때 풀이 생성됨)
    db = RDSClient()
    
    # 2. 쿼리 실행 예시 (SELECT)
    print("--- 사용자 조회 ---")
    users = db.execute("SELECT * FROM crawl_target_id LIMIT 3;")
    if users:
        for user in users:
            print(user)


if __name__ == "__main__":
    main()

DB HOST: musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com
DB USER: admin
DB PASSWORD: qkqajrwkrnrnrn9_
DB NAME: musinsa
DB PORT: 3306
✅ DB Engine (Pool) 생성 완료
--- 사용자 조회 ---
{'product_id': 70061, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}
{'product_id': 70063, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}
{'product_id': 70067, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}


In [ ]:
# !pip install tqdm

In [6]:
### 상품 id 크롤링

import requests
from tqdm import tqdm
import time
import csv

In [ ]:

# 헤더 설정 (브라우저 요청과 동일하게)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.musinsa.com/category/003?gf=M&sortCode=SALE_ONE_YEAR_COUNT"
}

all_ids = set()
page = 1
page_size = 60  # API에서 요청하는 size
total_pages_estimate = 1350  # 약 7만7천개 / 60 -> 대략 3167 페이지, 안전하게 1350

# CSV 파일 열기 (중간 저장 가능)
with open("musinsa_bottom_ids.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["goodsNo"])  # 헤더

    # tqdm 진행률 표시
    for page in tqdm(range(1, total_pages_estimate + 1), desc="상품 ID 수집"):
        url = f"https://api.musinsa.com/api2/dp/v1/plp/goods?gf=M&sortCode=SALE_ONE_YEAR_COUNT&category=003&size={page_size}&caller=CATEGORY&page={page}&seen=0&seenAds="
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"\nRequest Error at page {page}: {e}")
            break
        except ValueError:
            print(f"\nJSON Decode Error at page {page}")
            print(response.text[:200])
            break

        items = data.get("data", {}).get("list", [])
        if not items:
            print(f"\n마지막 페이지 도달: page={page}")
            break

        for item in items:
            goods_no = item.get("goodsNo")
            if goods_no and goods_no not in all_ids:
                all_ids.add(goods_no)
                writer.writerow([goods_no])

        time.sleep(0.25)  # 서버 부담 방지

print(f"\n총 수집된 상품 ID: {len(all_ids)}")
print("CSV 파일로 저장 완료: musinsa_bottom_ids.csv")
     

In [ ]:
# !pip install webdriver-manager

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv
import re
import json

In [17]:

# ---------------------------
# 공통 함수
# ---------------------------
def extract_int(text):
    return int(re.sub(r"[^0-9]", "", text)) if text else None

def extract_number(text):
    if text is None:
        return None
    if "만" in text:
        try:
            return str(int(float(text.replace("만", "")) * 10000))
        except:
            return text
    cleaned = re.sub(r"[^0-9]", "", text)
    return cleaned if cleaned != "" else ""

def clean_product_name(name):
    return re.sub(r"-.*", "", name).strip()

# ---------------------------
# 상품 ID 읽기
# ---------------------------
goods_ids = []
with open("musinsa_bottom_ids.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # 헤더 제거
    for row in reader:
        goods_ids.append(row[0])

# ---------------------------
# 인덱스 구간 입력
# ---------------------------
start_idx = int(input("시작 인덱스를 입력하세요 (예: 0): "))
end_idx = int(input("끝 인덱스를 입력하세요 (예: 5000): "))

# 유효성 체크
if start_idx < 0 or end_idx > len(goods_ids):
    raise ValueError("인덱스 구간이 상품 ID 리스트 범위를 벗어났습니다.")

target_ids = goods_ids[start_idx:end_idx]
print(f"총 {len(target_ids)}개의 상품 크롤링 예정")

# ---------------------------
# Selenium 드라이버 설정
# ---------------------------
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

results = []

# ---------------------------
# 크롤링 시작
# ---------------------------
print("크롤링 시작!")

for idx, gid in enumerate(target_ids, start=1):
    item_start = time.time()
    print(f"[{idx}/{len(target_ids)}] 상품 {gid} 처리 중...")

    url = f"https://www.musinsa.com/products/{gid}"
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ======================================================
    # (1) 상품명
    # ======================================================
    name_tag = soup.select_one("div[class*='GoodsName__Wrap'] span")
    product_name = clean_product_name(name_tag.get_text(strip=True)) if name_tag else None

    # ======================================================
    # (2) 브랜드명
    # ======================================================
    brand_tag = soup.select_one("div[class*='Brand__Wrap'] span[class*='BrandName']")
    brand = None
    if brand_tag:
        brand = re.sub(r"[0-9\.]+만", "", brand_tag.get_text(strip=True)).strip()

    # ======================================================
    # (3) 정가 / (4) 판매가 / (5) 할인율
    # ======================================================
    normal_tag = soup.select_one("span.line-through")
    normal_price = extract_int(normal_tag.get_text()) if normal_tag else None

    sale_tag = soup.select_one("span[class*='CalculatedPrice']")
    sale_price = extract_int(sale_tag.get_text()) if sale_tag else None

    discount_tag = soup.select_one("span[class*='DiscountRate']")
    discount = extract_int(discount_tag.get_text()) if discount_tag else None

    # ======================================================
    # (6) 카테고리
    # ======================================================
    cats = soup.select("div[class*='Category__Wrap'] a")
    upper_category = cats[0].get_text(strip=True) if len(cats) > 0 else None
    lower_category = cats[1].get_text(strip=True) if len(cats) > 1 else None

    # ======================================================
    # (7) 성별
    # ======================================================
    gender = 0
    layout_boxes = soup.select("dl[class*='Layout__Wrap'] div[class*='Layout__Box']")
    if len(layout_boxes) > 1:
        dd = layout_boxes[1].find("dd")
        if dd:
            gtext = dd.get_text(strip=True)
            gender_map = {"남": 1, "여": 2, "공용": 0, "남녀공용": 0}
            gender = gender_map.get(gtext, 0)

    # ---------------------------
    # 누적판매
    # ---------------------------
    cumulative = ""
    for box in layout_boxes:
        dt = box.find("dt")
        dd = box.find("dd")
        if not dt or not dd:
            continue
        if dt.get_text(strip=True) == "누적판매":
            cumulative = dd.get_text(strip=True)
            break

    # ======================================================
    # (8) 별점
    # ======================================================
    rating_tag = soup.select_one("div[class*='Review__Wrap'] span")
    rating = float(rating_tag.get_text(strip=True)) if rating_tag else None

    # ======================================================
    # (9) 후기수
    # ======================================================
    review_cnt_tag = soup.select_one("div[class*='Review__Wrap'] span:nth-of-type(2)")
    review_cnt = int(extract_number(review_cnt_tag.get_text())) if review_cnt_tag else 0

    # ======================================================
    # (10) 관심수
    # ======================================================
    like_tag = soup.select_one("div[class*='Like__Container'] span")
    like_cnt = extract_number(like_tag.get_text()) if like_tag else None

    # ======================================================
    # (11) 스타일 태그
    # ======================================================
    tag_items = soup.select("ul[class*='ProductTags__List'] span")
    styles = ",".join([t.get_text(strip=True) for t in tag_items])

    # ======================================================
    # (12) 실측사이즈 JSON
    # ======================================================
    size_json = ""
    header_ul = soup.select_one("ul[class*='ActualSizeHeaderUl']")
    size_table = soup.select_one("table[class*='ActualSizeTable']")

    if header_ul and size_table:
        header_lis = header_ul.select("li")
        size_names = [li.get_text(strip=True) for li in header_lis[2:]]

        header_cells = size_table.select("thead th")
        headers = [h.get_text(strip=True) for h in header_cells]

        rows = size_table.select("tbody tr")
        size_list = []
        row_index = 0

        for r in rows:
            tds = r.select("td")
            td_texts = [td.get_text(strip=True) for td in tds]

            if not any(char.isdigit() for char in "".join(td_texts)):
                continue

            size_dict = {"사이즈": size_names[row_index]}
            size_dict.update(dict(zip(headers, td_texts)))
            size_list.append(size_dict)
            row_index += 1

        size_json = json.dumps(size_list, ensure_ascii=False)

    # ======================================================
    # (13) 핏/계절감 JSON
    # ======================================================
    fit_json = {"핏": [], "계절감": []}
    mat_table = soup.select_one("table[class*='MaterialInfo__MaterialTable']")

    if mat_table:
        mat_rows = mat_table.select("tbody tr")

        for td in mat_rows[0].select("td"):
            if "cRPlvH" in td.get("class", []):
                fit_json["핏"].append(td.get_text(strip=True))

        for td in mat_rows[5].select("td"):
            if "cRPlvH" in td.get("class", []):
                fit_json["계절감"].append(td.get_text(strip=True))

    fit_json = json.dumps(fit_json, ensure_ascii=False)

    # ======================================================
    # 저장 리스트 추가
    # ======================================================
    results.append([
        gid, product_name, brand, normal_price, sale_price,
        upper_category, lower_category, gender, rating,
        like_cnt, review_cnt, size_json, discount,
        fit_json, cumulative, styles
    ])

driver.quit()

# ---------------------------
# CSV 저장
# ---------------------------
output_name = f"musinsa_bottom_{start_idx}_{end_idx}.csv"

with open(output_name, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "상품id","상품명","브랜드","정가","판매가",
        "상위카테고리","하위카테고리","성별","별점",
        "관심수","후기수","실측사이즈","할인율",
        "핏_계절감","누적판매","스타일"
    ])
    writer.writerows(results)

print(f"완료! → {output_name} 생성됨")


총 100개의 상품 크롤링 예정
크롤링 시작!
[1/100] 상품 1844582 처리 중...
[2/100] 상품 1926048 처리 중...
[3/100] 상품 2574822 처리 중...
[4/100] 상품 1551840 처리 중...
[5/100] 상품 3791988 처리 중...
[6/100] 상품 1926034 처리 중...
[7/100] 상품 2112061 처리 중...
[8/100] 상품 1551839 처리 중...
[9/100] 상품 1149329 처리 중...
[10/100] 상품 4274049 처리 중...
[11/100] 상품 750908 처리 중...
[12/100] 상품 2112059 처리 중...
[13/100] 상품 4757347 처리 중...
[14/100] 상품 1168906 처리 중...
[15/100] 상품 3228764 처리 중...
[16/100] 상품 3231055 처리 중...
[17/100] 상품 2578996 처리 중...
[18/100] 상품 3674341 처리 중...
[19/100] 상품 3663072 처리 중...
[20/100] 상품 1735427 처리 중...
[21/100] 상품 4993414 처리 중...
[22/100] 상품 2744549 처리 중...
[23/100] 상품 1924274 처리 중...
[24/100] 상품 2926541 처리 중...
[25/100] 상품 1436504 처리 중...
[26/100] 상품 1168922 처리 중...
[27/100] 상품 5012749 처리 중...
[28/100] 상품 4111296 처리 중...
[29/100] 상품 4556449 처리 중...
[30/100] 상품 5066602 처리 중...
[31/100] 상품 3575430 처리 중...
[32/100] 상품 5367961 처리 중...
[33/100] 상품 2513018 처리 중...
[34/100] 상품 3443300 처리 중...
[35/100] 상품 1627892 처리 중...
[36/